<a href="https://colab.research.google.com/github/sayashraaj/BTP/blob/main/Gurobi_trial_MDA3_WIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TO DO**



1. Big M using indicator variable DONE
2. Model is infeasible when t[origin] is used DONE
3. Convert DB to 14 Day DB
4. Build Cost from DB, not dummy
5. Change t[dest] to ? for multiple dests
6. Test on bigger sample network DONE
7. Use variability in sample network

### Pre Processing

In [47]:
# Importing the DataFrame

import pandas as pd
url = "https://raw.githubusercontent.com/sayashraaj/Dump/main/TrainScheduleDB.csv"
df = pd.read_csv(url, encoding='cp1252')

df.head(60)

,TrainId,TrainName,StationId,StationName,ScheduledArrival,ScheduledDeparture,Distance,Day
0,22981,KOTA - SRIGANGANAGAR SF EXPRESS,KOTA,KOTA JUNCTION,Source,17:20,0.0,1
1,22981,KOTA - SRIGANGANAGAR SF EXPRESS,KPTN,KESHORAI PATAN,17:35,17:37,13.3,1
2,22981,KOTA - SRIGANGANAGAR SF EXPRESS,LKE,LAKHERI,18:07,18:09,60.6,1
3,22981,KOTA - SRIGANGANAGAR SF EXPRESS,IDG,INDARGARH SUMERGANJ MANDI,18:18,18:20,71.4,1
4,22981,KOTA - SRIGANGANAGAR SF EXPRESS,SWM,SAWAI MADHOPUR JUNCTION,19:00,19:15,107.9,1
5,22981,KOTA - SRIGANGANAGAR SF EXPRESS,CKB,CHAUTH KA BARWARA,19:45,19:47,129.9,1
6,22981,KOTA - SRIGANGANAGAR SF EXPRESS,ISA,ISARDA,20:00,20:02,147.8,1
7,22981,KOTA - SRIGANGANAGAR SF EXPRESS,BNLW,BANASTHALI NIWAI,20:21,20:22,173.7,1
8,22981,KOTA - SRIGANGANAGAR SF EXPRESS,DPA,DURGAPURA,21:13,21:16,232.3,1
9,22981,KOTA - SRIGANGANAGAR SF EXPRESS,JP,JAIPUR JUNCTION,21:35,21:45,239.7,1


In [48]:
stations_list = ["WL", "TPJ", "MAS", "SBC", "NDLS", "MMCT"]
df_filtered = df[df["StationId"].isin(stations_list)]
df_filtered.head(5)

,TrainId,TrainName,StationId,StationName,ScheduledArrival,ScheduledDeparture,Distance,Day
46716,51901,AGRA CANTT. - OLD DELHI PASSENGER,NDLS,NEW DELHI,15:35,15:40,195.0,1
46721,51902,OLD DELHI - AGRA CANT. PASSENGER,NDLS,NEW DELHI,07:15,07:17,3.3,1
49478,1707,JABALPUR - ATARI FARE SPECIAL,NDLS,NEW DELHI,00:20,00:50,910.7,2
49491,1708,ATARI - JABALPUR FARE SPECIAL,NDLS,NEW DELHI,01:30,02:00,473.4,2
49684,2172,JAMMU TAWI - MUMBAI CST AC SF SPECIAL,NDLS,NEW DELHI,16:40,17:00,577.1,1


In [49]:
# Filtering df to have trains which stop at at least 1 station in the list

TrainId_dict = {}

for row in df_filtered.itertuples():
  TrainId_dict[row.TrainId] = []

TrainId_list = list(TrainId_dict.keys())
print("TrainId_List: ",TrainId_list)

df_filtered_1 = df[df["TrainId"].isin(TrainId_list)]
df_filtered_1.head(50)

TrainId_List:  ['51901', '51902', '1707', '1708', '2172', '2193', '2194', '2779', '4012', '4905', '4906', '6001', '6002', '6005', '6006', '6007', '6008', '6011', '6015', '6016', '6025', '6026', '6027', '6028', '6037', '6048', '6051', '6052', '6057', '6058', '6063', '6229', '6230', '6539', '6540', '6569', '6570', '6863', '6864', '7075', '7076', '7147', '7148', '7257', '7607', '7608', '8500', '9005', '9006', '11013', '11014', '11019', '11020', '11027', '11028', '11041', '11042', '11043', '11044', '11057', '11058', '11065', '11066', '11073', '11074', '11077', '11078', '11301', '11302', '11303', '11304', '11449', '11450', '11901', '11902', '12001', '12002', '12003', '12004', '12005', '12006', '12007', '12008', '12011', '12012', '12013', '12014', '12015', '12016', '12017', '12018', '12027', '12028', '12029', '12030', '12031', '12032', '12033', '12034', '12037', '12038', '12039', '12040', '12043', '12044', '12045', '12046', '12047', '12048', '12055', '12056', '12057', '12058', '12077', '1207

,TrainId,TrainName,StationId,StationName,ScheduledArrival,ScheduledDeparture,Distance,Day
46688,51901,AGRA CANTT. - OLD DELHI PASSENGER,AGC,AGRA CANTT.,Source,09:00,0.0,1
46689,51901,AGRA CANTT. - OLD DELHI PASSENGER,RKM,RAJA KI MANDI,09:09,09:10,4.0,1
46690,51901,AGRA CANTT. - OLD DELHI PASSENGER,BFP,BILOCHPURA AGRA,09:15,09:16,5.6,1
46691,51901,AGRA CANTT. - OLD DELHI PASSENGER,RNKA,RUNKUTA,09:25,09:26,14.6,1
46692,51901,AGRA CANTT. - OLD DELHI PASSENGER,KXM,KITHAM,09:36,09:37,23.6,1
46693,51901,AGRA CANTT. - OLD DELHI PASSENGER,FHT,FARAH TOWN,09:46,09:47,32.8,1
46694,51901,AGRA CANTT. - OLD DELHI PASSENGER,FAR,FARAH,09:52,09:53,35.1,1
46695,51901,AGRA CANTT. - OLD DELHI PASSENGER,BAD,BAAD,10:02,10:03,43.8,1
46696,51901,AGRA CANTT. - OLD DELHI PASSENGER,MTJ,MATHURA JUNCTION,10:38,10:46,53.8,1
46697,51901,AGRA CANTT. - OLD DELHI PASSENGER,BTSR,BHUTESHWAR,10:53,10:54,56.5,1


In [50]:
# Filtering dictionary to have trains which stop at at least 1 station in the list

for row in df_filtered_1.itertuples():
    if row.StationId in stations_list:
        TrainId_dict[row.TrainId].append(row.StationId)
#         if row.TrainId=="2193":
#             print(row.TrainId, row.ScheduledArrival, row.StationId)
# print(TrainId_dict["2193"])

print(len(TrainId_dict))

TrainId_dict_temp = {}
for TrainId in TrainId_dict:
    if len(TrainId_dict[TrainId])>1:
        TrainId_dict_temp[TrainId] = TrainId_dict[TrainId]
TrainId_dict = TrainId_dict_temp
print(len(TrainId_dict))

980
87


In [51]:
# Visual sanity check
for TrainId in TrainId_dict:
    print(TrainId, TrainId_dict[TrainId])

2193 ['TPJ', 'WL']
2194 ['WL', 'TPJ']
12007 ['MAS', 'SBC']
12008 ['SBC', 'MAS']
12027 ['MAS', 'SBC']
12028 ['SBC', 'MAS']
12295 ['SBC', 'MAS', 'WL']
12296 ['WL', 'MAS', 'SBC']
12433 ['MAS', 'WL']
12434 ['WL', 'MAS']
12511 ['WL', 'MAS']
12512 ['MAS', 'WL']
12521 ['WL', 'MAS']
12522 ['MAS', 'WL']
12577 ['WL', 'MAS', 'SBC']
12578 ['SBC', 'MAS', 'WL']
12607 ['MAS', 'SBC']
12608 ['SBC', 'MAS']
12609 ['MAS', 'SBC']
12610 ['SBC', 'MAS']
12615 ['MAS', 'WL', 'NDLS']
12616 ['NDLS', 'WL']
12621 ['MAS', 'WL', 'NDLS']
12622 ['NDLS', 'WL', 'MAS']
12625 ['WL', 'NDLS']
12626 ['NDLS', 'WL']
12627 ['SBC', 'NDLS']
12628 ['NDLS', 'SBC']
12639 ['MAS', 'SBC']
12640 ['SBC', 'MAS']
12655 ['WL', 'MAS']
12656 ['MAS', 'WL']
12657 ['MAS', 'SBC']
12658 ['SBC', 'MAS']
12669 ['MAS', 'WL']
12670 ['WL', 'MAS']
12687 ['MAS', 'WL']
12687-Slip ['MAS', 'WL']
12688 ['WL', 'MAS']
12689 ['MAS', 'TPJ']
12690 ['TPJ', 'MAS']
12691 ['MAS', 'SBC']
12692 ['SBC', 'MAS']
12759 ['MAS', 'WL']
12760 ['WL', 'MAS']
12851 ['WL', 'MAS']
12

In [52]:
# Filtering df again to have trains which stop at at least 2 stations in the list

TrainId_list = list(TrainId_dict.keys())
print("TrainId_List: ",TrainId_list)

df_filtered_1 = df[df["TrainId"].isin(TrainId_list)]
df_filtered_1.head(50)

TrainId_List:  ['2193', '2194', '12007', '12008', '12027', '12028', '12295', '12296', '12433', '12434', '12511', '12512', '12521', '12522', '12577', '12578', '12607', '12608', '12609', '12610', '12615', '12616', '12621', '12622', '12625', '12626', '12627', '12628', '12639', '12640', '12655', '12656', '12657', '12658', '12669', '12670', '12687', '12687-Slip', '12688', '12689', '12690', '12691', '12692', '12759', '12760', '12851', '12852', '12967', '12968', '12969', '12970', '15119', '15120', '16021', '16022', '16031', '16032', '16093', '16094', '16231', '16232', '16317', '16318', '16318-Slip', '16687', '16688', '16688-Slip', '16793', '16794', '22351', '22352', '22403', '22404', '22415', '22416', '22625', '22626', '22631', '22632', '22645', '22646', '22647', '22648', '22681', '22682', '16787', '22688']


,TrainId,TrainName,StationId,StationName,ScheduledArrival,ScheduledDeparture,Distance,Day
49743,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,TEN,TIRUNELVELI JUNCTION,Source,17:45,0.0,1
49744,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,CVP,KOVILPATTI,19:08,19:10,65.2,1
49745,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,SRT,SATUR,19:33,19:35,86.8,1
49746,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,VPT,VIRUDUNAGAR JUNCTION,20:08,20:10,113.7,1
49747,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,MDU,MADURAI JUNCTION,20:55,21:00,157.1,1
49748,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,DG,DINDIGUL JUNCTION,22:00,22:05,219.4,1
49749,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,TPJ,TIRUCHCHIRAPPALLI JUNCTION,00:05,00:10,313.8,2
49750,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,KRR,KARUR JUNCTION,01:30,01:32,390.0,2
49751,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,NMKL,NAMAKKAL,02:13,02:15,423.9,2
49752,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,RASP,RASIPURAM,02:40,02:42,449.6,2


In [61]:
df1 = df_filtered_1

# Risky
# df1 = df

In [62]:
# https://towardsdatascience.com/a-really-simple-way-to-edit-row-by-row-in-a-pandas-dataframe-75d339cbd313

def get_sec(time_str, day_str):
    """Get seconds from time."""
    h, m= time_str.split(':')
    return int(h)*3600 + int(m)*60 + int(day_str)*86400

for index in df1.index:
  if df1.loc[index,'ScheduledArrival']!='Source':
    df1.loc[index,'ScheduledArrival'] = get_sec(str(df1.loc[index,'ScheduledArrival']), str(df1.loc[index,'Day']))
  if df1.loc[index,'ScheduledDeparture']!='Destination':
    df1.loc[index,'ScheduledDeparture'] = get_sec(str(df1.loc[index,'ScheduledDeparture']), str(df1.loc[index,'Day']))

df1.head(50)

,TrainId,TrainName,StationId,StationName,ScheduledArrival,ScheduledDeparture,Distance,Day
49743,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,TEN,TIRUNELVELI JUNCTION,Source,150300,0.0,1
49744,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,CVP,KOVILPATTI,155280,155400,65.2,1
49745,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,SRT,SATUR,156780,156900,86.8,1
49746,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,VPT,VIRUDUNAGAR JUNCTION,158880,159000,113.7,1
49747,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,MDU,MADURAI JUNCTION,161700,162000,157.1,1
49748,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,DG,DINDIGUL JUNCTION,165600,165900,219.4,1
49749,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,TPJ,TIRUCHCHIRAPPALLI JUNCTION,173100,173400,313.8,2
49750,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,KRR,KARUR JUNCTION,178200,178320,390.0,2
49751,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,NMKL,NAMAKKAL,180780,180900,423.9,2
49752,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,RASP,RASIPURAM,182400,182520,449.6,2


In [63]:
# Declaring Node class

# class Node:
#   def __init__(self, TrainId, TrainName, StationId, StationName, ScheduledArrival, ScheduledDeparture, Day):
#     self.TrainId = TrainId
#     self.TrainName = TrainName
#     self.StationId = StationId
#     self.StationName = StationName
#     self.ScheduledArrival = ScheduledArrival
#     self.ScheduledDeparture = ScheduledDeparture
#     self.Day = Day
#     self.connectedTo = []

class Node:
  def __init__(self, TrainId, TrainName, StationId, StationName, Time, Day):
    self.TrainId = TrainId
    self.TrainName = TrainName
    self.StationId = StationId
    self.StationName = StationName
    self.Time = Time
    self.Day = Day
    self.connectedTo = []

In [64]:
# Creating and storing nodes in nodesList
nodesList = []

for row in df1.itertuples():
  if row.ScheduledArrival=='Source':
    # print("Source detected")
    continue
  else:
    nodesList.append(Node(row.TrainId, row.TrainName, row.StationId, row.StationName, row.ScheduledArrival, row.Day))
  if row.ScheduledDeparture=='Destination':
    # print("Destination detected")
    continue
  else:
    nodesList.append(Node(row.TrainId, row.TrainName, row.StationId, row.StationName, row.ScheduledDeparture, row.Day))

In [65]:
# Printing connectedTo []

def printConnectedTo(i):
  print("Node details", nodesList[i].TrainId, nodesList[i].Time, nodesList[i].Day)
  print("Printing connectedTo for Node",i)
  for x in nodesList[i].connectedTo:
    print("Connection:", x.TrainId, x.Time, x.Day)
  print("----------------------")

def printNodeDetails(node):
  print(node.TrainId, node.TrainName, node.StationId, node.StationName, node.Time, node.Day)

In [66]:
# # Linking for same Train

for i in range(len(nodesList)-1):
  j=i+1
  nodeA = nodesList[i]
  nodeB = nodesList[j]
  if i==j or nodeA.TrainId!=nodeB.TrainId or nodeA.Time>nodeB.Time:
    continue
  else:
    nodeA.connectedTo.append(nodeB)

In [67]:
# Connected nodes with same StationId

# for i in range(len(nodesList)):
#   for j in range(len(nodesList)):
#     nodeA = nodesList[i]
#     nodeB = nodesList[j]
#     if i==j or nodeA.StationId!=nodeB.StationId or nodeA.TrainId==nodeB.TrainId or nodeA.Time>nodeB.Time or nodeA.Day>nodeB.Day:
#       continue
#     else:
#       # print("connecting",nodeA.TrainId, nodeA.Time, nodeA.Day,"to", nodeB.TrainId, nodeB.Time, nodeA.Day)
#       nodeA.connectedTo.append(nodeB)

for i in range(len(nodesList)):
  for j in range(len(nodesList)):
    nodeA = nodesList[i]
    nodeB = nodesList[j]
    if i==j or nodeA.StationId!=nodeB.StationId or nodeA.TrainId==nodeB.TrainId or nodeA.Time>nodeB.Time:
      continue
    else:
      nodeA.connectedTo.append(nodeB)

In [68]:
# for i in range(len(nodesList)):
#   printConnectedTo(i)

In [69]:
# converting to nodes[]

nodes = []
nodesDict = {}
reverse_nodesDict = {}
for i in range(len(nodesList)):
  # Storing only the node number
  nodes.append(i)

  # Mapping Node -> i
  nodesDict[nodesList[i]]=i

  # Mapping i -> Node
  reverse_nodesDict[i]=nodesList[i]

print("Number of nodes: ",len(nodes))
print(nodes[19])

Number of nodes:  5318
19


In [70]:
# converting to links[(a,b)]
# node 29, 30 have multiple links

links = []

for i in range(len(nodesList)):
  for j in range(len(nodesList[i].connectedTo)):
    links.append((nodesDict[nodesList[i]], nodesDict[nodesList[i].connectedTo[j]]))

print("Number of links: ",len(links))
print(links[1024])

Number of links:  118863
(39, 3092)


In [71]:
# Costs
cost = [[0 for x in nodes] for y in nodes]

for (a,b) in links:
    cost[a][b]= reverse_nodesDict[b].Time-reverse_nodesDict[a].Time + 1

    # Checking for time violation in links
    if cost[a][b] < 0:
      print(cost[a][b], a, b)
      printNodeDetails(reverse_nodesDict[a])
      printNodeDetails(reverse_nodesDict[b])

### Setup and Installation

In [72]:
!pip install gurobipy  # install gurobipy, if not already installed
import gurobipy as gp  # import the installed package

# Create an environment with your WLS license
params = {
"WLSACCESSID": 'f5a52c8d-c747-487a-bf9e-fd3f23ec1c72',
"WLSSECRET": '0bff086e-4c28-4979-b1d9-eadc994a435f',
"LICENSEID": 942624,
}
env = gp.Env(params=params)

# Create the model within the Gurobi environment
model = gp.Model(env=env)

from gurobipy import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 942624
Academic license - for non-commercial use only - registered to ce19b113@smail.iitm.ac.in


In [73]:
def forwardStarMaker(arr):
  ans = {}
  for (a,b) in arr:
    if a in ans:
      ans[a].append(b)
    else:
      ans[a] = [b]

  for n in nodes:
    if n not in ans:
      ans[n]=[]
  return ans

def backwardStarMaker(arr):
  ans = {}
  for (a,b) in arr:
    if b in ans:
      ans[b].append(a)
    else:
      ans[b] = [a]
  for n in nodes:
    if n not in ans:
      ans[n]=[]

  return ans

def printTvalues(t, x, nodes, links):
  ans = {}
  for (a,b) in links:
    if x[a,b].x:
      print('Flow on link ', (a, b), ' is:', x[a, b].x)
      ans[a]=1
      ans[b]=1
  print("***********Printing T Values**************")
  ans_T_dict = {}
  for n in ans.keys():
    ans_T_dict[t[n].x]=n
  for time_index, node_index in sorted(ans_T_dict.items()):
    print(time_index,":",node_index,": ",reverse_nodesDict[node_index].TrainId, reverse_nodesDict[node_index].TrainName, reverse_nodesDict[node_index].Time, reverse_nodesDict[node_index].Day, reverse_nodesDict[node_index].StationName)

In [74]:
# print(backwardStarMaker(links))
def stationSetArrayMaker(station_name):
  stationSetArray = []
  for node in nodesList:
    if node.StationId == str(station_name):
      # print(nodesDict[node], node.Time)
      stationSetArray.append(nodesDict[node])
  return stationSetArray

### Multiple Dest Approach 2

In [100]:
# LP trial

def gurobi_optimiser_multiple_dest_2(origin_set, possible_destinations_set, nodesToVisit_set):

  N = len(nodes)

  m = gp.Model(env=env) # Use this for Gurobi Academic License
  t = m.addVars(nodes, vtype = GRB.INTEGER)
  x = m.addVars(links, vtype = GRB.BINARY)

  forwardStar = forwardStarMaker(links)
  backwardStar = backwardStarMaker(links)

  m.addConstr(sum(sum([x[j,origin] for j in backwardStar[origin]]) for origin in origin_set) ==0 , name="origin IN")
  m.addConstr(sum(sum([x[possible_destination, j] for j in forwardStar[possible_destination]]) for possible_destination in possible_destinations_set) == 0, name="dests OUT")

  m.addConstr(sum(sum([x[origin,j] for j in forwardStar[origin]]) for origin in origin_set) >=1 , name="origin OUT")
  m.addConstr(sum(sum([x[j, possible_destination] for j in backwardStar[possible_destination]]) for possible_destination in possible_destinations_set) >= 1, name="dests IN")


  for (i,j) in links:
  #   # Original Indicator constraints
    m.addConstr((x[i,j] == 1) >> (t[j]-t[i]>=1), name = "t[i] t[j] LB activity time ["+str(i)+", "+str(j)+"]")
    # m.addConstr((x[i,j] == 1) >> (t[j]-t[i]<=cost[i][j]+1000), name = "t[i] t[j] UB activity time ["+str(i)+", "+str(j)+"]")

  for i in nodes:
      if i not in origin_set and i not in possible_destinations_set:
        m.addConstr(sum([x[i, j] for j in forwardStar[i]]) == sum([x[j, i] for j in backwardStar[i]]), name = "flow conservation ["+str(i)+", "+str(j)+"]")

  for n2v_set in nodesToVisit_set:
    m.addConstr(sum(sum([x[j, i] for j in backwardStar[i]]) for i in n2v_set) >= 1, name="intermediate IN, "+str(i))


  obj = sum([cost[i][j]*x[i,j] for (i,j) in links])

  m.Params.outputFlag = 0
  m.setObjective(obj, sense = GRB.MINIMIZE)
  m.optimize()

  if m.status == GRB.OPTIMAL:
      print('model is optimal ✅✅✅')
      print('objective value:',obj.getValue())
      # print('objective value:',obj.x)
      print('')
      # for (a,b) in links:
      #   if b in nodesToVisit:
      #     print(a,b, x[a,b].x)
      #   if a in nodesToVisit:
      #     print(a,b, x[a,b].x)
      printTvalues(t, x, nodes, links)
  elif m.status == GRB.INFEASIBLE:
      print('model is infeasible ❌❌❌')

      # To print the constraint violations:
      # m.computeIIS();
      # for c in m.getConstrs():
      #     if c.IISConstr:
      #         print('%s' % c.constrName)
  elif m.status == GRB.UNBOUNDED:
      print('model is unbounded')
  else:
      print('Not sure')
  
  return m

In [102]:
places_To_Vist = ["WL", "BZA"]
# places_To_Vist = ["SBC"]
# places_To_Vist = []

origin_set = stationSetArrayMaker("NDLS")
possible_destinations_set = stationSetArrayMaker("MAS")

nodesToVisit_set = []

for place_To_Visit in places_To_Vist:
  nodesToVisit_set.append(stationSetArrayMaker(place_To_Visit))

gurobi_optimiser_multiple_dest_2(origin_set, possible_destinations_set, nodesToVisit_set)

model is optimal ✅✅✅
objective value: 131421.0

Flow on link  (1154, 1155)  is: 1.0
Flow on link  (1155, 1503)  is: 1.0
Flow on link  (1251, 1252)  is: 1.0
Flow on link  (1252, 1253)  is: 1.0
Flow on link  (1253, 1254)  is: 1.0
Flow on link  (1254, 1255)  is: 1.0
Flow on link  (1255, 1256)  is: 1.0
Flow on link  (1256, 1257)  is: 1.0
Flow on link  (1257, 1258)  is: 1.0
Flow on link  (1258, 1259)  is: 1.0
Flow on link  (1259, 1260)  is: 1.0
Flow on link  (1260, 1261)  is: 1.0
Flow on link  (1261, 1262)  is: 1.0
Flow on link  (1262, 1263)  is: 1.0
Flow on link  (1263, 1264)  is: 1.0
Flow on link  (1264, 1265)  is: 1.0
Flow on link  (1265, 1266)  is: 1.0
Flow on link  (1266, 1267)  is: 1.0
Flow on link  (1267, 1268)  is: 1.0
Flow on link  (1503, 1504)  is: 1.0
Flow on link  (1504, 1251)  is: 1.0
***********Printing T Values**************
-0.0 : 1154 :  12616 GRAND TRUNK  153600 1 NEW DELHI
1.0 : 1155 :  12616 GRAND TRUNK  160800 1 MATHURA JUNCTION
4.0 : 1503 :  12628 KARNATAKA SF EXPRESS 

<gurobi.Model MIP instance Unnamed: 5237 constrs, 124181 vars, Parameter changes: LicenseID=942624, OutputFlag=0>

In [83]:
printNodeDetails(reverse_nodesDict[2162])

12692 SATYA SAI PRASHANTI NILAYAM - CHENNAI CENTRAL EXPRESS  SBC KSR BENGALURU CITY JUNCTION 169200 1
